# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [160]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [161]:
# define path

file_path='../WeatherPy/Output_data/cities.csv'
cities_df=pd.read_csv(file_path)
cities_df.describe()
#no NA value but have extra colume 	Unnamed: 0

,Unnamed: 0,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Date
count,563.000000,563.000000,563.000000,563.000000,563.000000,563.000000,563.000000,5.630000e+02
mean,281.000000,20.425460,17.644255,70.694565,66.033748,56.573712,8.048455,1.624284e+09
std,162.668374,33.429304,90.591251,15.770725,22.842193,38.809960,5.597009,1.388113e+02
min,0.000000,-54.800000,-175.200000,33.550000,4.000000,0.000000,0.000000,1.624283e+09
25%,140.500000,-8.054050,-63.984350,57.720000,53.000000,14.500000,3.760000,1.624284e+09
50%,281.000000,24.687700,20.866700,73.420000,72.000000,71.000000,7.000000,1.624284e+09
75%,421.500000,49.164000,102.518550,82.140000,83.000000,95.000000,11.600000,1.624284e+09
max,562.000000,78.218600,179.316700,113.560000,100.000000,100.000000,30.600000,1.624284e+09


In [162]:
# drop Unnamed: 0 colume
cities_df2=cities_df.drop(['Unnamed: 0'],1)
cities_df2.head()


,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,paita,-5.0892,-81.1144,66.76,80,36,11.83,PE,1624283497
1,pisco,-13.7000,-76.2167,64.45,77,90,4.61,PE,1624283499
2,punta arenas,-53.1500,-70.9167,41.11,75,75,14.97,CL,1624283207
3,ushuaia,-54.8000,-68.3000,49.66,53,40,5.75,AR,1624283328
4,husavik,66.0449,-17.3389,53.51,56,99,7.87,IS,1624283500


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [163]:
gmaps.configure(api_key=g_key)

In [164]:
# locations data
locations = cities_df2[['Lat','Lng']]

#  humidity data
humidity = cities_df2["Humidity"]

In [168]:
fig = gmaps.figure()

heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating = False, max_intensity = 100,point_radius = 3)

fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [60]:
# narrow down with restriction from read me description
cities_df_narrow=cities_df2[(cities_df2['Max Temp'] < 80) &
                             (cities_df2['Max Temp'] > 70) &
                             (cities_df2['Wind Speed'] < 10) &
                             (cities_df2['Cloudiness'] == 0)]

cities_df_clean=cities_df_narrow.dropna(how='any')
cities_df_clean.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
106,papetoai,-17.4961,-149.8729,72.99,74,0,0.98,PF,1624283577
197,troyes,48.3333,4.0833,77.05,50,0,4.61,FR,1624283646
292,luanda,-8.8368,13.2343,78.80,65,0,9.22,AO,1624283688
298,namibe,-15.1961,12.1522,75.83,69,0,8.05,AO,1624283725
560,durban,-29.8579,31.0292,75.92,69,0,3.00,ZA,1624283924


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [152]:
hotel_df=pd.DataFrame({"City":cities_df_clean['City'],
                     "Country":cities_df_clean['Country'],
                       "Lat":cities_df_clean['Lat'],
                       "Lng":cities_df_clean['Lng'],
                     "Hotel Name":""}
                     )
hotel_df

,City,Country,Lat,Lng,Hotel Name
106,papetoai,PF,-17.4961,-149.8729,
197,troyes,FR,48.3333,4.0833,
292,luanda,AO,-8.8368,13.2343,
298,namibe,AO,-15.1961,12.1522,
560,durban,ZA,-29.8579,31.0292,


In [154]:
url="https://maps.googleapis.com/maps/api/place/nearbysearch/json"
for i in range(len(cities_df_clean)) : 
   
    lat=str(cities_df_clean.iloc[i, 1])    
    lng=str(cities_df_clean.iloc[i, 2])
    location=lat+','+lng
    
    params = {"radius" : 5000,
          "key" : g_key,
          "type" : "hotel",
          "keyword" : "Hotel",
         "location":location}
    response=requests.get(url,params=params).json()
    hotel_df.iloc[i,4]=response['results'][0]['name']
    
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
106,papetoai,PF,-17.4961,-149.8729,Hilton Moorea Lagoon Resort & Spa
197,troyes,FR,48.3333,4.0833,Best Western Premier Hôtel De La Poste Troyes ...
292,luanda,AO,-8.8368,13.2343,EPIC SANA Luanda
298,namibe,AO,-15.1961,12.1522,iu Hotel Namibe
560,durban,ZA,-29.8579,31.0292,Southern Sun Elangeni & Maharani


In [155]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [156]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(markers)


# Display figure
fig

Figure(layout=FigureLayout(height='420px'))